# core

> Here we define the core functions that will be used in ERA5 repo.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def describe():
    print("This package fetches ERA5 data.")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()